In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install openai
!pip install tiktoken
import openai
import tiktoken
encoding = tiktoken.get_encoding("gpt2")
with open('/content/drive/MyDrive/DS-Environment-Project/climate-change-api-key.txt', 'r') as file:
    key = file.readline()
openai.api_key = key

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 3.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 22.0 MB/s eta 0:00:00


In [4]:
import json
import time
def batch_gpt(prompt,target_texts, batch_size):
    l = len(target_texts)
    size = int(l / batch_size) + 1
    size = batch_size
    text_list = np.array_split(target_texts, 3)
    print(f"Total records {l}, number of chunks = {size}")
    rez_keys = []
    rez_vals = []
    rez = []
    for i, texts in enumerate(text_list):
        text = "\n".join(texts)
        p = prompt + text
        print("prompt", p)
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": p}]
        )
        r = response["choices"][0]["message"]["content"]
        print(result, r)
        rez.append(r)
        try:
            dictData= json.loads(r)
            keys = list(dictData.keys())
            values = list(dictData.values())
            rez_keys += keys
            rez_vals += values
        except:
            print("error parsing"+r)
        print(f"i={i + 1}: shape:{len(rez_keys)}")
    return rez_keys, rez_vals, rez
def batch_gpt_len(prompt,target_texts, batch_size):
    l = len(target_texts)
    rez_keys = []
    rez_vals = []
    rez = []
    i =0
    while i < l:
        text = ""
        for j in range(batch_size):
            if i<l:
                if batch_size > 1:
                    text += f"{i}. "+target_texts[i]+"\n\n"
                else:
                    text += target_texts[i]
            i += 1
        p = prompt + text
        #print(f"Prompt ({len(p)}):", p)
        # print(i, len(p))

        # try the API call and if it fails, wait 10 seconds and retry again (max 3 times)
        for j in range(6):
            try:
                response = openai.ChatCompletion.create(
                    model="gpt-3.5-turbo",
                    messages=[{"role": "user", "content": p}]
                )
                break
            except:
                print("error calling API, retrying...")
                time.sleep(10)

        r = response["choices"][0]["message"]["content"]
        rez.append(r)
        # convert r to int
        try:
            b = int(r[:1])
        except:
            b = -1
            print("error parsing"+r)
        print(i, len(p), "-", r, b, end=": ")
        # if i % 10 print
        if i % 10 == 0:
            print()
        rez_keys.append(b)
    return rez,rez_keys


def batch_gpt_chained(prompt,chained_prompt,target_texts, batch_size):
    l = len(target_texts)
    rez_keys = []
    rez_vals = []
    rez = []
    i =0
    while i < l:
        text = ""
        for j in range(batch_size):
            if i<l:
                if batch_size > 1:
                    text += f"{i}. "+target_texts[i]+"\n\n"
                else:
                    text += target_texts[i]
            i += 1
        p = prompt + text
        #print(f"Prompt ({len(p)}):", p)
        # print(i, len(p))

        # try the API call and if it fails, wait 10 seconds and retry again (max 3 times)
        for j in range(6):
            try:
                response = openai.ChatCompletion.create(
                    model="gpt-3.5-turbo",
                    messages=[{"role": "user", "content": p}]
                )
                break
            except:
                print("error calling API, retrying...")
                time.sleep(10)

        r1 = response["choices"][0]["message"]["content"]
        try:
            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[{"role": "user", "content": chained_prompt + text}]
            )
        except:
            print("error calling API, retrying...")
            time.sleep(10)

        r = response["choices"][0]["message"]["content"]
        rez.append(r)
        # convert r to int
        try:
            b = int(r[:1])
        except:
            b = -1
            print("error parsing"+r)
        print(i, len(p), "-", r, b, end=": ")
        # if i % 10 print
        if i % 10 == 0:
            print()
        rez_keys.append(b)
    return rez,rez_keys



def batch_gpt_resp_as_part_prompt(prompt,chained_prompt,target_texts, batch_size):
    l = len(target_texts)
    rez_keys = []
    rez_vals = []
    rez = []
    summarized_points = []
    i =0
    while i < l:
        text = ""
        for j in range(batch_size):
            if i<l:
                if batch_size > 1:
                    text += f"{i}. "+target_texts[i]+"\n\n"
                else:
                    text += target_texts[i]
            i += 1
        p = prompt + text
        #print(f"Prompt ({len(p)}):", p)
        # print(i, len(p))

        # try the API call and if it fails, wait 10 seconds and retry again (max 3 times)
        for j in range(6):
            try:
                response = openai.ChatCompletion.create(
                    model="gpt-3.5-turbo",
                    messages=[{"role": "user", "content": p}]
                )
                break
            except:
                print("error calling API, retrying...")
                time.sleep(10)

        r1 = response["choices"][0]["message"]["content"]
        summarized_points.append(r1)
        try:
            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[{"role": "user", "content": chained_prompt + r1}]
            )
        except:
            print("error calling API, retrying...")
            time.sleep(10)

        r = response["choices"][0]["message"]["content"]
        rez.append(r)
        # convert r to int
        try:
            b = int(r[:1])
        except:
            b = -1
            print("error parsing"+r)
        print(i, len(p), "-", r, b, end=": ")
        # if i % 10 print
        if i % 10 == 0:
            print()
        rez_keys.append(b)
    return rez,rez_keys, summarized_points

# Trying to give ChatGPT the text first to analyze it and after that ask ChatGPT to classify it

# testing Climate Detection on ChatGPT
## Classification classes:
### 0 - is not climate change-related
### 1 - is climate change-related

In [ ]:
!pip install datasets
from datasets import load_dataset

dataset = load_dataset("climatebert/climate_detection")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 14.4 MB/s eta 0:00:00


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/1300 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/400 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/climatebert___parquet/climatebert--climate_detection-eefa7af3c8031d26/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 1300
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 400
    })
})

In [ ]:
dataset['test']

Dataset({
    features: ['text', 'label'],
    num_rows: 400
})

In [ ]:
data = []
for i in range(0,len(dataset['test']['text'])):
  data.append([dataset['test']['text'][i],dataset['test']['label'][i]])

print(data)

[['Sustainable strategy ‘red lines’ For our sustainable strategy range, we incorporate a series of proprietary ‘red lines’ in order to ensure the poorest- performing companies from an ESG perspective are not eligible for investment.', 1], ['Verizon’s environmental, health and safety management system provides a framework for identifying, controlling, and reducing the risks associated with the environments in which we operate. Besides regular management system assessments, internal and third-party compliance audits and inspections are performed annually at hundreds of facilities worldwide. The goal of these assessments is to identify and correct site-specific issues, and to educate and empower facility managers and supervisors to implement corrective actions. Verizon’s environment, health and safety efforts are directed and supported by experienced experts around the world that support our operations and facilities.', 1], ['In 2019, the Company closed a series of transactions related to

In [ ]:
import pandas as pd
df = pd.DataFrame(data=data,columns=["text","label"])

In [ ]:
df

text  label
0    Sustainable strategy ‘red lines’ For our susta...      1
1    Verizon’s environmental, health and safety man...      1
2    In 2019, the Company closed a series of transa...      1
3    In December 2020, the AUC approved the Electri...      1
4    Finally, there is a reputational risk linked t...      1
..                                                 ...    ...
395  In 2020, Banco do Brasil Foundation celebrated...      1
396  Climate change is producing changes in weather...      1
397  A sound and certain regulatory and fiscal envi...      0
398  Across our global workforce, 20% of Gold Field...      0
399  The purchase price of these raw materials and ...      0

[400 rows x 2 columns]

In [ ]:
prompt = 'You are the sustainability, environment, and climate change expert. Read the following text about sustainability, the environment, or climate change and analyze it as an expert: \n\n'
chain_prompt = 'Now you need to classify the analyzed text whether the text is about sustainability, the environment, or climate change. Answer only with 1 if the text is sustainability, environment or climate change related or 0 if not: \n\n'
texts = df["text"].to_list()
rez, rez_keys = batch_gpt_chained(prompt,chain_prompt, texts, 1)

1 409 - 1 (sustainability related) 1: 2 867 - 1 (environment related) 1: 3 796 - 0 0: 4 1109 - 0 0: 5 599 - 1 (climate change related) 1: 6 568 - 1 (environment and sustainability related) 1: 7 565 - 1 (climate change related) 1: 8 613 - 1 (sustainability related) 1: 9 709 - 1 (Environment and Climate Change related) 1: 10 484 - 1 (climate change related) 1: 
11 773 - 0 0: error calling API, retrying...
error parsingAs an AI language model, I do not hold the expertise of a sustainability, environment, and climate change expert. However, the given text highlights the significance of climate change and its implications for the insurance industry. It indicates that businesses are acknowledging the importance of understanding and addressing climate-related risks and opportunities. This is crucial as climate change can have long-term effects on business operations, making it necessary for companies to integrate sustainable practices. Thus, it is encouraging to see businesses recognizing the

In [ ]:
rez

['1 (sustainability related)',
 '1 (environment related)',
 '0',
 '0',
 '1 (climate change related)',
 '1 (environment and sustainability related)',
 '1 (climate change related)',
 '1 (sustainability related)',
 '1 (Environment and Climate Change related)',
 '1 (climate change related)',
 '0',
 'As an AI language model, I do not hold the expertise of a sustainability, environment, and climate change expert. However, the given text highlights the significance of climate change and its implications for the insurance industry. It indicates that businesses are acknowledging the importance of understanding and addressing climate-related risks and opportunities. This is crucial as climate change can have long-term effects on business operations, making it necessary for companies to integrate sustainable practices. Thus, it is encouraging to see businesses recognizing the need for sustainability and taking proactive measures to manage climate change risks.',
 '1 (sustainability related)',
 '0

In [ ]:
rez_keys

[1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 -1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 -1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 -1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 -1,
 1,
 -1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,

In [ ]:
df['gpt-label'] = rez_keys

In [ ]:
df['gpt-explanations'] = rez

In [ ]:
df

text  label  gpt-label  \
0    Sustainable strategy ‘red lines’ For our susta...      1          1   
1    Verizon’s environmental, health and safety man...      1          1   
2    In 2019, the Company closed a series of transa...      1          0   
3    In December 2020, the AUC approved the Electri...      1          0   
4    Finally, there is a reputational risk linked t...      1          1   
..                                                 ...    ...        ...   
395  In 2020, Banco do Brasil Foundation celebrated...      1          1   
396  Climate change is producing changes in weather...      1          1   
397  A sound and certain regulatory and fiscal envi...      0          0   
398  Across our global workforce, 20% of Gold Field...      0          0   
399  The purchase price of these raw materials and ...      0          0   

               gpt-explanations  
0    1 (sustainability related)  
1       1 (environment related)  
2                             0  
3                             0  
4    1 (climate change related)  
..                          ...  
395  1 (sustainability related)  
396  1 (climate change related)  
397                           0  
398                           0  
399                           0  

[400 rows x 4 columns]

In [ ]:
df['gpt-label'].value_counts()

 1    288
 0    105
-1      7
Name: gpt-label, dtype: int64

In [ ]:
df[df['gpt-label']==-1]

text  label  gpt-label  \
11   Climate-related risks and opportunities have g...      1         -1   
22    To exclude the least energy efficient hydroc...      1         -1   
52   We also look at the way banks facilitate finan...      1         -1   
97   New products include our $1 per day electric v...      1         -1   
99   The scenarios developed by the IEA were used a...      1         -1   
301  Although climate change has not yet had a mate...      1         -1   
323  As we invest in Manufactured Capital to improv...      1         -1   

                                      gpt-explanations  
11   As an AI language model, I do not hold the exp...  
22                                   Climate change: 1  
52   As a sustainability, environment, and climate ...  
97   As a sustainability expert, I can see that AGL...  
99   As a sustainability, environment, and climate ...  
301  operations and across the financial sector, an...  
323       and Social capitals. 1 (Environment related)

In [ ]:
df['gpt-label'] = df['gpt-label'].replace(-1,1)

In [ ]:
df['gpt-label'].value_counts()

1    295
0    105
Name: gpt-label, dtype: int64

In [ ]:
df.to_csv("/content/drive/MyDrive/DS-Environment-Project/ChatGPT Results/chatgpt_climate_detection_chained_prompts.csv",index=False)

In [ ]:
# calculate the precision and f1 score for df columns label and prediction
from sklearn.metrics import precision_recall_fscore_support,f1_score
sent_col = 'gpt-label'
print(precision_recall_fscore_support(df['label'], df[sent_col], average='macro'))

# f1 score only
print(f1_score(df['label'], df[sent_col], average='macro'))

# confusion matrix
from sklearn.metrics import confusion_matrix
print(confusion_matrix(df['label'], df[sent_col]))

# performnce report

from sklearn.metrics import classification_report
print(classification_report(df['label'], df[sent_col]))

(0.8163841807909604, 0.8828125, 0.8417930125247197, None)
0.8417930125247197
[[ 70  10]
 [ 35 285]]
              precision    recall  f1-score   support

           0       0.67      0.88      0.76        80
           1       0.97      0.89      0.93       320

    accuracy                           0.89       400
   macro avg       0.82      0.88      0.84       400
weighted avg       0.91      0.89      0.89       400



# testing Climate Sentiment on ChatGPT by using summarization of most important points in the paragraphs
## Classification classes:
### 0 - risk
### 1 - neutral
### 2 - opportunity

In [ ]:
!pip install datasets
from datasets import load_dataset

dataset = load_dataset("climatebert/climate_sentiment")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 26.7 MB/s eta 0:00:00


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/320 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/climatebert___parquet/climatebert--climate_sentiment-cde94fbdb0f4004a/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 320
    })
})

In [ ]:
dataset['test']

Dataset({
    features: ['text', 'label'],
    num_rows: 320
})

In [ ]:
data = []
for i in range(0,len(dataset['test']['text'])):
  data.append([dataset['test']['text'][i],dataset['test']['label'][i]])

print(data)

[['Sustainable strategy ‘red lines’ For our sustainable strategy range, we incorporate a series of proprietary ‘red lines’ in order to ensure the poorest- performing companies from an ESG perspective are not eligible for investment.', 0], ['Verizon’s environmental, health and safety management system provides a framework for identifying, controlling, and reducing the risks associated with the environments in which we operate. Besides regular management system assessments, internal and third-party compliance audits and inspections are performed annually at hundreds of facilities worldwide. The goal of these assessments is to identify and correct site-specific issues, and to educate and empower facility managers and supervisors to implement corrective actions. Verizon’s environment, health and safety efforts are directed and supported by experienced experts around the world that support our operations and facilities.', 1], ['In 2019, the Company closed a series of transactions related to

In [ ]:
import pandas as pd
df = pd.DataFrame(data=data,columns=["text","label"])

In [ ]:
df

text  label
0    Sustainable strategy ‘red lines’ For our susta...      0
1    Verizon’s environmental, health and safety man...      1
2    In 2019, the Company closed a series of transa...      1
3    In December 2020, the AUC approved the Electri...      0
4    Finally, there is a reputational risk linked t...      0
..                                                 ...    ...
315  Indirect emissions result from operational act...      1
316  All data in this TCFD report is as of, or for ...      1
317  Outcome: The bank explained that it would be w...      1
318  In 2020, Banco do Brasil Foundation celebrated...      2
319  Climate change is producing changes in weather...      0

[320 rows x 2 columns]

In [ ]:
prompt = 'You are the sustainability, environment, and climate change expert. Read the following paragraph and extract the most important points from the text and return only the points and their explanations: \n\n'
chain_prompt = 'Read the following points and answer only with the overall sentiment of all points summarized without any explanations. Answer only with 0 if the sentiment is risk related to sustainabilty, environment and climate change, answer only with 1 if the sentiment is neutral related to sustainabilty, environment and climate change and answer only with 2 if the sentiment is opportunity related to sustainabilty, environment and climate change: \n\n'
texts = df["text"].to_list()
rez, rez_keys = batch_gpt_resp_as_part_prompt(prompt,chain_prompt, texts, 1)

1 431 - 2 2: 2 889 - 1 1: 3 818 - 0 (Risk related to sustainability, environment and climate change) 0: 4 1131 - 1 1: 5 621 - 0 (risk related to sustainability, environment and climate change) 0: 6 590 - 2 2: 7 587 - 0 (risk related to sustainability, environment, and climate change) 0: 8 731 - 2 2: 9 506 - 1 1: error parsingThe overall sentiment of the points is 0 - risk related to sustainability, environment and climate change.
10 462 - The overall sentiment of the points is 0 - risk related to sustainability, environment and climate change. -1: 
11 474 - 2 2: 12 428 - 1 1: error parsingSentiment: 2
13 487 - Sentiment: 2 -1: 14 648 - 1 1: 15 571 - 1 1: 16 402 - 1 1: 17 878 - 0 0: 18 637 - 2 2: 19 913 - 2 2: 20 858 - 0 0: 
21 813 - 0 0: 22 564 - 0 0: 23 513 - 2 2: 24 453 - 2 2: 25 684 - 2 2: 26 816 - 0 0: 27 814 - 0 0: 28 411 - 1 1: 29 510 - 2 2: 30 705 - 0 0: 
31 869 - 2 (neutral) 2: error calling API, retrying...
32 1039 - 2 2: 33 794 - 2 (opportunity related to sustainability, envi

In [ ]:
rez

['2',
 '1',
 '0 (Risk related to sustainability, environment and climate change)',
 '1',
 '0 (risk related to sustainability, environment and climate change)',
 '2',
 '0 (risk related to sustainability, environment, and climate change)',
 '2',
 '1',
 'The overall sentiment of the points is 0 - risk related to sustainability, environment and climate change.',
 '2',
 '1',
 'Sentiment: 2',
 '1',
 '1',
 '1',
 '0',
 '2',
 '2',
 '0',
 '0',
 '0',
 '2',
 '2',
 '2',
 '0',
 '0',
 '1',
 '2',
 '0',
 '2 (neutral)',
 '2',
 '2 (opportunity related to sustainability, environment and climate change)',
 '2',
 '0',
 '2',
 '0',
 '1. Reducing energy consumption and GHG emissions\n- This is important to help reduce the amount of GHG emissions released into the atmosphere, which can contribute to climate change.\n- It can also help to mitigate potential impacts from policy or regulatory actions that may result in increased costs for carbon emissions.\n\n2. Specific actions being taken\n- It is mentioned that

In [ ]:
rez_keys

[2,
 1,
 0,
 1,
 0,
 2,
 0,
 2,
 1,
 -1,
 2,
 1,
 -1,
 1,
 1,
 1,
 0,
 2,
 2,
 0,
 0,
 0,
 2,
 2,
 2,
 0,
 0,
 1,
 2,
 0,
 2,
 2,
 2,
 2,
 0,
 2,
 0,
 1,
 2,
 2,
 2,
 2,
 0,
 1,
 0,
 2,
 1,
 2,
 0,
 2,
 2,
 0,
 1,
 1,
 2,
 2,
 2,
 0,
 1,
 2,
 2,
 2,
 2,
 0,
 0,
 2,
 0,
 1,
 2,
 0,
 1,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 1,
 1,
 0,
 0,
 0,
 1,
 2,
 2,
 2,
 -1,
 2,
 1,
 2,
 2,
 0,
 1,
 2,
 2,
 0,
 0,
 1,
 1,
 1,
 2,
 1,
 1,
 2,
 -1,
 2,
 1,
 2,
 0,
 2,
 2,
 2,
 1,
 2,
 1,
 1,
 1,
 2,
 2,
 0,
 2,
 2,
 1,
 0,
 2,
 1,
 2,
 0,
 0,
 0,
 2,
 2,
 1,
 2,
 2,
 2,
 2,
 1,
 2,
 2,
 2,
 2,
 0,
 0,
 2,
 1,
 2,
 2,
 2,
 2,
 2,
 1,
 1,
 1,
 2,
 1,
 2,
 -1,
 2,
 2,
 0,
 1,
 1,
 0,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 2,
 0,
 1,
 2,
 0,
 2,
 1,
 1,
 2,
 0,
 2,
 2,
 2,
 2,
 0,
 1,
 2,
 2,
 0,
 2,
 2,
 1,
 0,
 2,
 2,
 1,
 2,
 0,
 0,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 0,
 2,
 -1,
 1,
 2,
 2,
 2,
 0,
 1,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 1,
 1,
 2,
 2,
 0,
 0,
 2,
 2,
 2,
 2,
 1,
 2,
 1

In [ ]:
df['gpt-label'] = rez_keys

In [ ]:
df['gpt-explanations'] = rez

In [ ]:
df

text  label  gpt-label  \
0    Sustainable strategy ‘red lines’ For our susta...      0          2   
1    Verizon’s environmental, health and safety man...      1          1   
2    In 2019, the Company closed a series of transa...      1          0   
3    In December 2020, the AUC approved the Electri...      0          1   
4    Finally, there is a reputational risk linked t...      0          0   
..                                                 ...    ...        ...   
315  Indirect emissions result from operational act...      1          1   
316  All data in this TCFD report is as of, or for ...      1          1   
317  Outcome: The bank explained that it would be w...      1          2   
318  In 2020, Banco do Brasil Foundation celebrated...      2          2   
319  Climate change is producing changes in weather...      0          1   

                                      gpt-explanations  
0                                                    2  
1                                                    1  
2    0 (Risk related to sustainability, environment...  
3                                                    1  
4    0 (risk related to sustainability, environment...  
..                                                 ...  
315                                                  1  
316                                                  1  
317                                                  2  
318                                                  2  
319                                                  1  

[320 rows x 4 columns]

In [ ]:
df['gpt-label'].value_counts()

 2    167
 1     77
 0     69
-1      7
Name: gpt-label, dtype: int64

In [ ]:
df[df['gpt-label']==-1]

text  label  gpt-label  \
9    Climate-related risks and opportunities have g...      0         -1   
12   Moreover, we continue to enhance the durabilit...      2         -1   
87   In 2020, we contracted three operations pursua...      1         -1   
105  The number of reports remains limited thanks t...      1         -1   
158  We recognise that there is strong empirical ev...      1         -1   
217  Certification Acquisition Status The DNP Group...      1         -1   
293  We determine which stores are included in the ...      0         -1   

                                      gpt-explanations  
9    The overall sentiment of the points is 0 - ris...  
12                                        Sentiment: 2  
87                                Overall sentiment: 1  
105                                       Sentiment: 2  
158                               Overall sentiment: 2  
217  Overall sentiment: 2 (opportunity related to s...  
293  - Comp sales calculation for stores is determi...

In [ ]:
df['gpt-label'] = df['gpt-label'].replace(-1,2)

In [ ]:
df['gpt-label'].value_counts()

2    174
1     77
0     69
Name: gpt-label, dtype: int64

In [ ]:
df.to_csv("/content/drive/MyDrive/DS-Environment-Project/ChatGPT Results/chatgpt_climate_sentiment_chained_prompts.csv",index=False)

In [ ]:
# calculate the precision and f1 score for df columns label and prediction
from sklearn.metrics import precision_recall_fscore_support,f1_score
sent_col = 'gpt-label'
print(precision_recall_fscore_support(df['label'], df[sent_col], average='macro'))

# f1 score only
print(f1_score(df['label'], df[sent_col], average='macro'))

# confusion matrix
from sklearn.metrics import confusion_matrix
print(confusion_matrix(df['label'], df[sent_col]))

# performnce report

from sklearn.metrics import classification_report
print(classification_report(df['label'], df[sent_col]))

(0.5697757182015053, 0.5868859640163508, 0.4878571428571428, None)
0.4878571428571428
[[ 55  25  26]
 [ 14  49 100]
 [  0   3  48]]
              precision    recall  f1-score   support

           0       0.80      0.52      0.63       106
           1       0.64      0.30      0.41       163
           2       0.28      0.94      0.43        51

    accuracy                           0.48       320
   macro avg       0.57      0.59      0.49       320
weighted avg       0.63      0.47      0.48       320



## adding the summarized points to scikit-llm library

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/DS-Environment-Project/ChatGPT Results/chatgpt_climate_tcfd_chained_prompts.csv")

In [ ]:
df_sent = pd.read_csv("/content/drive/MyDrive/IntroToDataScience/DS-Environment-Project/ChatGPT Results/chatgpt_climate_sentiment.csv")

In [ ]:
df

text  label  gpt-label  \
0    Sustainable strategy ‘red lines’ For our susta...      2          2   
1    Verizon’s environmental, health and safety man...      3          3   
2    In 2019, the Company closed a series of transa...      2          0   
3    In December 2020, the AUC approved the Electri...      2          4   
4    Finally, there is a reputational risk linked t...      2          3   
..                                                 ...    ...        ...   
395  In 2020, Banco do Brasil Foundation celebrated...      2          1   
396  Climate change is producing changes in weather...      2          1   
397  A sound and certain regulatory and fiscal envi...      0          3   
398  Across our global workforce, 20% of Gold Field...      0          1   
399  The purchase price of these raw materials and ...      0          0   

                            gpt-explanations  \
0                                          2   
1                                          3   
2                                          0   
3                                          4   
4                                          3   
..                                       ...   
395                                        1   
396                                        1   
397  to investment \n0 (not climate-related)   
398                                        1   
399                                        0   

                                                points  
0    Points:\n1. Sustainable strategy range\n2. Pro...  
1    Points:\n1. Verizon has an environmental, heal...  
2    Points:\n- The Company sold its Canadian fossi...  
3    Points:\n- AUC approved deferral of distributi...  
4    - Reputational risk for oil companies \n- Poss...  
..                                                 ...  
395  Points:\n- Banco do Brasil Foundation celebrat...  
396  1. Climate change leads to changes in environm...  
397  Points:\n1. Regulatory and fiscal environment ...  
398  - Gold Fields' global workforce has 20% women ...  
399  Key Points: \n\n- Raw material and fuel prices...  

[400 rows x 5 columns]

In [ ]:
df['label'] = df_sent['label']

In [ ]:
df = df[:320]

In [ ]:
df['label'] = df_sent['label']

<ipython-input-83-b5fc60e5602d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = df_sent['label']


In [ ]:
df

text  label  gpt-label  \
0    Sustainable strategy ‘red lines’ For our susta...      0          2   
1    Verizon’s environmental, health and safety man...      1          2   
2    In 2019, the Company closed a series of transa...      1          2   
3    In December 2020, the AUC approved the Electri...      0          1   
4    Finally, there is a reputational risk linked t...      0          0   
..                                                 ...    ...        ...   
315  Energy Infrastructure revenues of $195 million...      1          1   
316  Financial planning We acknowledge that there a...      1          2   
317  : PCG Group has adopted the PETRONAS Whistlebl...      1          1   
318  Stakeholder dialogue and transparency ⌐ Engagi...      2          2   
319  Foreign, federal, state and local regulatory a...      0          0   

                                      gpt-explanations  \
0    the sentiment of the provided text is opportun...   
1    the sentiment of the provided text is opportun...   
2    the sentiment of the provided text is opportun...   
3    the sentiment of the provided text is neutral ...   
4    the sentiment of the provided text is risk abo...   
..                                                 ...   
315  the sentiment of the provided text is neutral ...   
316  the sentiment of the provided text is opportun...   
317  the sentiment of the provided text is neutral ...   
318  the sentiment of the provided text is opportun...   
319  the sentiment of the provided text is risk abo...   

                                                points  
0    Points:\n1. Sustainable strategy range\n2. Pro...  
1    Points:\n1. Verizon has an environmental, heal...  
2    Points:\n- The Company sold its Canadian fossi...  
3    Points:\n- AUC approved deferral of distributi...  
4    - Reputational risk for oil companies \n- Poss...  
..                                                 ...  
315  - Energy Infrastructure revenues in 2020 were ...  
316  Points:\n-Company intends to integrate climate...  
317  Points:\n- PCG Group has adopted the PETRONAS ...  
318  - Adidas engages openly with stakeholders and ...  
319  Points:\n\n1. Proposed legislation and regulat...  

[320 rows x 5 columns]

In [ ]:
from skllm import ZeroShotGPTClassifier

X = df['points']
Y = df['label']


clf = ZeroShotGPTClassifier()
clf.fit(None, ["the sentiment of the provided text is risk about sustainability, environment and climate change","the sentiment of the provided text is neutral about sustainability, environment and climate change","the sentiment of the provided text is opportunity about sustainability, environment and climate change"])
preds = clf.predict(X)

100%|██████████| 320/320 [07:58<00:00,  1.49s/it]


In [ ]:
preds

['the sentiment of the provided text is opportunity about sustainability, environment and climate change',
 'the sentiment of the provided text is opportunity about sustainability, environment and climate change',
 'the sentiment of the provided text is opportunity about sustainability, environment and climate change',
 'the sentiment of the provided text is neutral about sustainability, environment and climate change',
 'the sentiment of the provided text is risk about sustainability, environment and climate change',
 'the sentiment of the provided text is opportunity about sustainability, environment and climate change',
 'the sentiment of the provided text is risk about sustainability, environment and climate change',
 'the sentiment of the provided text is opportunity about sustainability, environment and climate change',
 'the sentiment of the provided text is opportunity about sustainability, environment and climate change',
 'the sentiment of the provided text is opportunity abo

In [ ]:
df['gpt-explanations'] = preds

<ipython-input-87-263adef27334>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['gpt-explanations'] = preds


In [ ]:
df[df['gpt-explanations'] == "the sentiment of the provided text is risk about sustainability, environment and climate change"]

text  label  gpt-label  \
4    Finally, there is a reputational risk linked t...      0          0   
6    The Group and its customers are exposed to cli...      0          0   
11   Climate-related risks and opportunities have g...      2          0   
13   An emerging risk is defined as a risk which ha...      1          0   
17   We emit greenhouse gases both directly and ind...      1          0   
..                                                 ...    ...        ...   
302  Nedbank has a Damage to Physical Assets scenar...      1          0   
310  Rising public concern related to climate chang...      1          0   
311  In addition, certain environmental laws assess...      1          0   
312  However, we believe that the data currently di...      1          0   
319  Foreign, federal, state and local regulatory a...      0          0   

                                      gpt-explanations  \
4    the sentiment of the provided text is risk abo...   
6    the sentiment of the provided text is risk abo...   
11   the sentiment of the provided text is risk abo...   
13   the sentiment of the provided text is risk abo...   
17   the sentiment of the provided text is risk abo...   
..                                                 ...   
302  the sentiment of the provided text is risk abo...   
310  the sentiment of the provided text is risk abo...   
311  the sentiment of the provided text is risk abo...   
312  the sentiment of the provided text is risk abo...   
319  the sentiment of the provided text is risk abo...   

                                                points  
4    - Reputational risk for oil companies \n- Poss...  
6    1. The Group and its customers are exposed to ...  
11   Points:\n\n1. Climate-related risks and opport...  
13   Points:\n\n1. Emerging risk is defined as a ri...  
17   Points:\n\n- Greenhouse gases are emitted both...  
..                                                 ...  
302  Points:\n\n- Nedbank has a scenario for physic...  
310  Points:\n1. Rising public concern related to c...  
311  1. Environmental laws can hold current or prev...  
312  Points: \n- Data disclosed by companies does n...  
319  Points:\n\n1. Proposed legislation and regulat...  

[74 rows x 5 columns]

In [ ]:
labels = []

for label in preds:
  if label == "the sentiment of the provided text is risk about sustainability, environment and climate change":
    labels.append(0)
  elif label == "the sentiment of the provided text is neutral about sustainability, environment and climate change":
    labels.append(1)
  elif label == "the sentiment of the provided text is opportunity about sustainability, environment and climate change":
    labels.append(2)


labels

[2,
 2,
 2,
 1,
 0,
 2,
 0,
 2,
 2,
 2,
 1,
 0,
 1,
 0,
 2,
 2,
 2,
 0,
 2,
 1,
 1,
 2,
 2,
 0,
 0,
 0,
 2,
 2,
 2,
 0,
 0,
 2,
 2,
 0,
 1,
 1,
 2,
 2,
 0,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 1,
 2,
 2,
 1,
 2,
 0,
 2,
 2,
 1,
 1,
 1,
 2,
 2,
 2,
 0,
 0,
 2,
 2,
 1,
 2,
 2,
 2,
 0,
 1,
 0,
 2,
 0,
 0,
 2,
 2,
 0,
 1,
 0,
 2,
 1,
 2,
 1,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 0,
 0,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 1,
 2,
 2,
 0,
 2,
 2,
 0,
 1,
 1,
 0,
 2,
 2,
 0,
 2,
 0,
 1,
 1,
 2,
 2,
 1,
 2,
 2,
 1,
 2,
 1,
 0,
 0,
 1,
 2,
 2,
 2,
 1,
 2,
 0,
 0,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 0,
 2,
 1,
 1,
 2,
 0,
 0,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 0,
 0,
 2,
 1,
 2,
 2,
 2,
 2,
 1,
 2,
 0,
 0,
 1,
 1,
 1,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 1,
 1,
 2,
 0,
 2,
 2,
 2,
 0,
 2,
 2,
 1,
 2,
 1,
 0,
 2,
 1,
 0,
 0,
 0,
 2,
 1,
 2,
 0,
 1,
 1,
 0,
 2,
 2,
 1,
 2,
 1,
 2,
 1,
 1,
 2,
 2,
 2,
 2,
 1,
 2,
 2,
 1,
 1,
 2,
 0,
 0,
 2,
 1,
 0,
 1,
 2,
 2,
 2,
 2,
 0,
 2,
 1,
 2,
 1,
 2,
 2,


In [ ]:
df['gpt-label'] = labels

<ipython-input-90-bd7b018092de>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['gpt-label'] = labels


In [ ]:
df

text  label  gpt-label  \
0    Sustainable strategy ‘red lines’ For our susta...      0          2   
1    Verizon’s environmental, health and safety man...      1          2   
2    In 2019, the Company closed a series of transa...      1          2   
3    In December 2020, the AUC approved the Electri...      0          1   
4    Finally, there is a reputational risk linked t...      0          0   
..                                                 ...    ...        ...   
315  Energy Infrastructure revenues of $195 million...      1          1   
316  Financial planning We acknowledge that there a...      1          2   
317  : PCG Group has adopted the PETRONAS Whistlebl...      1          1   
318  Stakeholder dialogue and transparency ⌐ Engagi...      2          2   
319  Foreign, federal, state and local regulatory a...      0          0   

                                      gpt-explanations  \
0    the sentiment of the provided text is opportun...   
1    the sentiment of the provided text is opportun...   
2    the sentiment of the provided text is opportun...   
3    the sentiment of the provided text is neutral ...   
4    the sentiment of the provided text is risk abo...   
..                                                 ...   
315  the sentiment of the provided text is neutral ...   
316  the sentiment of the provided text is opportun...   
317  the sentiment of the provided text is neutral ...   
318  the sentiment of the provided text is opportun...   
319  the sentiment of the provided text is risk abo...   

                                                points  
0    Points:\n1. Sustainable strategy range\n2. Pro...  
1    Points:\n1. Verizon has an environmental, heal...  
2    Points:\n- The Company sold its Canadian fossi...  
3    Points:\n- AUC approved deferral of distributi...  
4    - Reputational risk for oil companies \n- Poss...  
..                                                 ...  
315  - Energy Infrastructure revenues in 2020 were ...  
316  Points:\n-Company intends to integrate climate...  
317  Points:\n- PCG Group has adopted the PETRONAS ...  
318  - Adidas engages openly with stakeholders and ...  
319  Points:\n\n1. Proposed legislation and regulat...  

[320 rows x 5 columns]

In [ ]:
df['gpt-label'].value_counts()

2    179
0     74
1     67
Name: gpt-label, dtype: int64

In [ ]:
df.to_csv("/content/drive/MyDrive/DS-Environment-Project/ChatGPT Results/scikit-llm/chatgpt_climate_sentiment_chained_prompt.csv",index=False)

In [ ]:
# calculate the precision and f1 score for df columns label and prediction
from sklearn.metrics import precision_recall_fscore_support,f1_score
sent_col = 'gpt-label'
print(precision_recall_fscore_support(df['label'], df[sent_col], average='macro'))

# f1 score only
print(f1_score(df['label'], df[sent_col], average='macro'))

# confusion matrix
from sklearn.metrics import confusion_matrix
print(confusion_matrix(df['label'], df[sent_col]))

# performnce report

from sklearn.metrics import classification_report
print(classification_report(df['label'], df[sent_col]))

(0.3103037582734072, 0.2989660810112524, 0.24879227053140096, None)
0.24879227053140096
[[21 21 64]
 [39 34 90]
 [14 12 25]]
              precision    recall  f1-score   support

           0       0.28      0.20      0.23       106
           1       0.51      0.21      0.30       163
           2       0.14      0.49      0.22        51

    accuracy                           0.25       320
   macro avg       0.31      0.30      0.25       320
weighted avg       0.37      0.25      0.26       320



# testing Climate Disclosure classification on ChatGPT by using summarization of most important points in the paragraphs
## Classification classes:
### 0 - not climate-related
### 1 - metrics
### 2 - strategy
### 3 - risk
### 4 - governance

In [ ]:
!pip install datasets
from datasets import load_dataset

dataset = load_dataset("climatebert/tcfd_recommendations")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/1300 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/400 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/climatebert___parquet/climatebert--tcfd_recommendations-8f7123f770abbd61/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 1300
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 400
    })
})

In [ ]:
dataset['test']

Dataset({
    features: ['text', 'label'],
    num_rows: 400
})

In [ ]:
data = []
for i in range(0,len(dataset['test']['text'])):
  data.append([dataset['test']['text'][i],dataset['test']['label'][i]])

print(data)

[['Sustainable strategy ‘red lines’ For our sustainable strategy range, we incorporate a series of proprietary ‘red lines’ in order to ensure the poorest- performing companies from an ESG perspective are not eligible for investment.', 2], ['Verizon’s environmental, health and safety management system provides a framework for identifying, controlling, and reducing the risks associated with the environments in which we operate. Besides regular management system assessments, internal and third-party compliance audits and inspections are performed annually at hundreds of facilities worldwide. The goal of these assessments is to identify and correct site-specific issues, and to educate and empower facility managers and supervisors to implement corrective actions. Verizon’s environment, health and safety efforts are directed and supported by experienced experts around the world that support our operations and facilities.', 3], ['In 2019, the Company closed a series of transactions related to

In [ ]:
import pandas as pd
df = pd.DataFrame(data=data,columns=["text","label"])

In [ ]:
df

text  label
0    Sustainable strategy ‘red lines’ For our susta...      2
1    Verizon’s environmental, health and safety man...      3
2    In 2019, the Company closed a series of transa...      2
3    In December 2020, the AUC approved the Electri...      2
4    Finally, there is a reputational risk linked t...      2
..                                                 ...    ...
395  In 2020, Banco do Brasil Foundation celebrated...      2
396  Climate change is producing changes in weather...      2
397  A sound and certain regulatory and fiscal envi...      0
398  Across our global workforce, 20% of Gold Field...      0
399  The purchase price of these raw materials and ...      0

[400 rows x 2 columns]

In [ ]:
prompt = 'You are the sustainability, environment, and climate change expert. Read the following paragraph and extract the most important points from the text and return only the points and their explanations: \n\n'
chain_prompt = 'Read the following points and answer only with the overall class of which all points are summarized without any explanations. Answer only with 0 if the text is not climate-related, answer only with 1 if the text is about metrics for sustainability, environment, and climate change, answer only with 2 if the text is about strategy for sustainability, environment, and climate change, answer only with 3 if the text is about risk for sustainability, environment, and climate change and answer only with 4 if the text is about governance for sustainability, environment, and climate change: \n\n'
texts = df["text"].to_list()
summarized_points = []
rez, rez_keys, summarized_points = batch_gpt_resp_as_part_prompt(prompt,chain_prompt, texts, 1)

1 431 - 2 2: 2 889 - 3 3: 3 818 - 0 0: 4 1131 - 4 4: 5 621 - 3 3: 6 590 - 1 1: 7 587 - 3 3: 8 635 - 1. 0
2. 0
3. 1 1: 9 731 - 4 4: 10 506 - 1 1: 
11 795 - 4 4: 12 462 - 3 3: 13 474 - 1 1: 14 708 - 3 (risk for sustainability, environment, and climate change) 3: 15 428 - 1 1: 16 637 - 0 0: 17 487 - 1 1: 18 648 - 1 1: 19 571 - 3 3: 20 402 - 3 3: 
21 878 - 3 3: 22 637 - 0 0: 23 913 - 1 (metrics for sustainability, environment, and climate change) 1: 24 858 - 1 1: 25 813 - 3 3: error calling API, retrying...
error parsing- Point: Increased severity/frequency of extreme weather events
- Explanation: Refers to the worsening and more frequent occurrence of natural disasters such as hurricanes, floods, and wildfires.

- Point: Loss or impairment of key manufacturing sites
- Explanation: Refers to the damage or destruction of facilities where products are manufactured, leading to disruptions in supply chains and production.

- Point: Inability to procure sufficient raw materials
- Explanation: R

In [ ]:
rez

['2',
 '3',
 '0',
 '4',
 '3',
 '1',
 '3',
 '1. 0\n2. 0\n3. 1',
 '4',
 '1',
 '4',
 '3',
 '1',
 '3 (risk for sustainability, environment, and climate change)',
 '1',
 '0',
 '1',
 '1',
 '3',
 '3',
 '3',
 '0',
 '1 (metrics for sustainability, environment, and climate change)',
 '1',
 '3',
 '- Point: Increased severity/frequency of extreme weather events\n- Explanation: Refers to the worsening and more frequent occurrence of natural disasters such as hurricanes, floods, and wildfires.\n\n- Point: Loss or impairment of key manufacturing sites\n- Explanation: Refers to the damage or destruction of facilities where products are manufactured, leading to disruptions in supply chains and production.\n\n- Point: Inability to procure sufficient raw materials\n- Explanation: Refers to the difficulty in obtaining necessary resources due to disruptions in transportation or other barriers.\n\n- Point: Disruption to transportation of raw materials or finished goods\n- Explanation: Refers to difficulties

In [ ]:
rez_keys

[2,
 3,
 0,
 4,
 3,
 1,
 3,
 1,
 4,
 1,
 4,
 3,
 1,
 3,
 1,
 0,
 1,
 1,
 3,
 3,
 3,
 0,
 1,
 1,
 3,
 -1,
 2,
 1,
 4,
 3,
 0,
 2,
 2,
 3,
 2,
 1,
 -1,
 2,
 3,
 1,
 3,
 1,
 2,
 2,
 2,
 1,
 4,
 3,
 4,
 1,
 4,
 1,
 2,
 1,
 1,
 2,
 3,
 4,
 4,
 1,
 1,
 2,
 3,
 3,
 1,
 2,
 4,
 3,
 1,
 4,
 3,
 0,
 1,
 4,
 3,
 3,
 0,
 2,
 3,
 0,
 1,
 1,
 0,
 2,
 4,
 2,
 2,
 2,
 3,
 2,
 0,
 3,
 1,
 3,
 3,
 3,
 1,
 1,
 1,
 1,
 4,
 1,
 0,
 1,
 1,
 4,
 3,
 4,
 1,
 3,
 3,
 0,
 3,
 1,
 4,
 1,
 3,
 3,
 0,
 0,
 4,
 1,
 4,
 1,
 2,
 3,
 3,
 3,
 1,
 3,
 4,
 2,
 1,
 3,
 2,
 2,
 1,
 3,
 4,
 2,
 1,
 3,
 3,
 3,
 2,
 1,
 3,
 4,
 4,
 1,
 3,
 3,
 3,
 3,
 1,
 1,
 2,
 1,
 2,
 4,
 1,
 4,
 1,
 1,
 3,
 2,
 2,
 3,
 3,
 1,
 4,
 2,
 2,
 1,
 1,
 3,
 1,
 3,
 3,
 3,
 4,
 0,
 4,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 3,
 1,
 1,
 2,
 4,
 1,
 1,
 4,
 1,
 4,
 2,
 4,
 3,
 4,
 1,
 1,
 3,
 3,
 1,
 1,
 4,
 1,
 4,
 3,
 3,
 1,
 4,
 0,
 3,
 3,
 1,
 4,
 4,
 1,
 2,
 1,
 1,
 0,
 2,
 3,
 1,
 4,
 1,
 3,
 3,
 1,
 4,
 3,
 4,
 4,
 1,
 4,
 2,
 3,
 2,
 4,
 2,
 3,
 4,
 4

In [ ]:
df['gpt-label'] = rez_keys

In [ ]:
df['gpt-explanations'] = rez

In [ ]:
df

text  label  gpt-label  \
0    Sustainable strategy ‘red lines’ For our susta...      2          2   
1    Verizon’s environmental, health and safety man...      3          3   
2    In 2019, the Company closed a series of transa...      2          0   
3    In December 2020, the AUC approved the Electri...      2          4   
4    Finally, there is a reputational risk linked t...      2          3   
..                                                 ...    ...        ...   
395  In 2020, Banco do Brasil Foundation celebrated...      2          1   
396  Climate change is producing changes in weather...      2          1   
397  A sound and certain regulatory and fiscal envi...      0         -1   
398  Across our global workforce, 20% of Gold Field...      0          1   
399  The purchase price of these raw materials and ...      0          0   

                            gpt-explanations  
0                                          2  
1                                          3  
2                                          0  
3                                          4  
4                                          3  
..                                       ...  
395                                        1  
396                                        1  
397  to investment \n0 (not climate-related)  
398                                        1  
399                                        0  

[400 rows x 4 columns]

In [ ]:
df['gpt-label'].value_counts()

 1    126
 3    110
 2     65
 4     58
 0     36
-1      5
Name: gpt-label, dtype: int64

In [ ]:
df[df['gpt-label']==-1]

text  label  gpt-label  \
25   2 Increased severity/frequency of extreme weat...      2         -1   
36   BB DTVM, by strategic direction, performs exte...      3         -1   
259  At Group level, in 2019 there were 61 spills o...      3         -1   
374  Report a description of how their investment p...      2         -1   
397  A sound and certain regulatory and fiscal envi...      0         -1   

                                      gpt-explanations  
25   - Point: Increased severity/frequency of extre...  
36   Class 2: Metrics for sustainability, environme...  
259  - In 2019, there were 61 spills of hazardous m...  
374                                   Overall class: 1  
397            to investment \n0 (not climate-related)

In [ ]:
df['gpt-label'] = df['gpt-label'].replace(-1,3)

In [ ]:
df['gpt-label'].value_counts()

1    126
3    115
2     65
4     58
0     36
Name: gpt-label, dtype: int64

In [ ]:
summarized_points

["Points:\n1. Sustainable strategy range\n2. Proprietary 'red lines'\n3. Ensuring poorest-performing companies not eligible for investment\n\nExplanation:\n1. The text mentions a sustainable strategy range, which suggests that the company in question has a focus on sustainability and is implementing strategies to achieve this goal.\n2. The company uses proprietary 'red lines,' or criteria, to determine which companies are eligible for investment. These criteria likely focus on environmental, social, and governance (ESG) performance, which are important markers of sustainability.\n3. By excluding the poorest-performing companies from investment, the company is prioritizing sustainability and sending a message that it takes ESG factors seriously. This can help drive positive change within the broader investment community and promote better practices among companies.",
 "Points:\n1. Verizon has an environmental, health, and safety management system \n2. The system identifies, controls, an

In [ ]:
df['points'] = summarized_points

In [ ]:
df.to_csv("/content/drive/MyDrive/DS-Environment-Project/ChatGPT Results/chatgpt_climate_tcfd_chained_prompts.csv",index=False)

In [ ]:
# calculate the precision and f1 score for df columns label and prediction
from sklearn.metrics import precision_recall_fscore_support,f1_score
sent_col = 'gpt-label'
print(precision_recall_fscore_support(df['label'], df[sent_col], average='macro'))

# f1 score only
print(f1_score(df['label'], df[sent_col], average='macro'))

# confusion matrix
from sklearn.metrics import confusion_matrix
print(confusion_matrix(df['label'], df[sent_col]))

# performnce report

from sklearn.metrics import classification_report
print(classification_report(df['label'], df[sent_col]))

(0.4178209063966185, 0.4800612603495127, 0.3666315515630334, None)
0.3666315515630334
[[22  8  3 28 19]
 [ 1 39  6  2  1]
 [13 69 45 53 17]
 [ 0  8  6 27  7]
 [ 0  2  5  5 14]]
              precision    recall  f1-score   support

           0       0.61      0.28      0.38        80
           1       0.31      0.80      0.45        49
           2       0.69      0.23      0.34       197
           3       0.23      0.56      0.33        48
           4       0.24      0.54      0.33        26

    accuracy                           0.37       400
   macro avg       0.42      0.48      0.37       400
weighted avg       0.54      0.37      0.36       400



## trying to give the summarized points to scikit-llm library

In [ ]:
!pip install scikit-llm

with open('/content/drive/MyDrive/DS-Environment-Project/climate-change-api-key.txt', 'r') as file1:
    key = file1.readline()

with open('/content/drive/MyDrive/DS-Environment-Project/climate-change-organization-key.txt', 'r') as file2:
    org_key = file2.readline()

from skllm.config import SKLLMConfig

SKLLMConfig.set_openai_key(key)
SKLLMConfig.set_openai_org(org_key)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.5/647.5 kB 11.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for annoy: filename=annoy-1.17.3-cp310-cp310-linux_x86_64.whl size=582699 sha256=64cc328da80c7478b71f938545c756f763d11711866dfd7f75b5f5b7d1fc746b
  Stored in directory: /root/.cache/pip/wheels/64/8a/da/f714bcf46c5efdcfcac0559e63370c21abe961c48e3992465a
Successfully built annoy


In [ ]:
from skllm import ZeroShotGPTClassifier

X = df['points']
Y = df['label']

candidate_labels = [
    "the provided text is not about sustainability, environment and climate change",
    "the provided text is about metrics for sustainability, environment and climate change",
    "the provided text is about strategy for sustainability, environment and climate change",
    "the provided text is about risk for sustainability, environment and climate change",
    "the provided text is about governance for sustainability, environment and climate change"
]


clf = ZeroShotGPTClassifier()
clf.fit(None, [candidate_labels])
preds = clf.predict(X)

100%|██████████| 400/400 [10:42<00:00,  1.61s/it]


In [ ]:
preds

['the provided text is about strategy for sustainability, environment and climate change',
 'the provided text is about risk for sustainability, environment and climate change',
 'the provided text is not about sustainability, environment and climate change',
 'the provided text is not about sustainability, environment and climate change',
 'the provided text is about risk for sustainability, environment and climate change',
 'the provided text is about metrics for sustainability, environment and climate change',
 'the provided text is about risk for sustainability, environment and climate change',
 'the provided text is not about sustainability, environment and climate change',
 'the provided text is about governance for sustainability, environment and climate change',
 'the provided text is about risk for sustainability, environment and climate change',
 'the provided text is not about sustainability, environment and climate change',
 'the provided text is about risk for sustainabili

In [ ]:
df['gpt-explanations'] = preds

In [ ]:
labels = []

for label in preds:
  if label == "the provided text is not about sustainability, environment and climate change":
    labels.append(0)
  elif label == "the provided text is about metrics for sustainability, environment and climate change":
    labels.append(1)
  elif label == "the provided text is about strategy for sustainability, environment and climate change":
    labels.append(2)
  elif label == "the provided text is about risk for sustainability, environment and climate change":
    labels.append(3)
  elif label == "the provided text is about governance for sustainability, environment and climate change":
    labels.append(4)


labels

[2,
 3,
 0,
 0,
 3,
 1,
 3,
 0,
 4,
 3,
 0,
 3,
 1,
 3,
 0,
 0,
 0,
 1,
 3,
 3,
 3,
 0,
 3,
 1,
 3,
 3,
 1,
 1,
 4,
 3,
 1,
 3,
 2,
 3,
 1,
 1,
 1,
 4,
 3,
 2,
 3,
 1,
 2,
 1,
 1,
 2,
 4,
 3,
 1,
 0,
 4,
 1,
 4,
 3,
 1,
 2,
 3,
 1,
 0,
 1,
 1,
 2,
 3,
 3,
 0,
 2,
 0,
 3,
 1,
 3,
 3,
 0,
 3,
 0,
 3,
 3,
 1,
 2,
 3,
 0,
 3,
 1,
 0,
 1,
 0,
 2,
 1,
 2,
 3,
 1,
 0,
 3,
 1,
 3,
 3,
 3,
 1,
 1,
 0,
 1,
 4,
 2,
 0,
 0,
 2,
 2,
 3,
 2,
 1,
 3,
 3,
 0,
 3,
 0,
 1,
 3,
 3,
 3,
 0,
 0,
 2,
 1,
 0,
 0,
 2,
 3,
 3,
 0,
 3,
 3,
 4,
 1,
 1,
 3,
 1,
 2,
 1,
 3,
 4,
 2,
 1,
 3,
 3,
 3,
 3,
 3,
 3,
 4,
 4,
 1,
 3,
 3,
 3,
 3,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 4,
 3,
 1,
 3,
 1,
 1,
 3,
 3,
 0,
 1,
 0,
 2,
 2,
 1,
 0,
 1,
 3,
 3,
 3,
 4,
 0,
 4,
 1,
 1,
 2,
 0,
 1,
 2,
 0,
 0,
 1,
 3,
 1,
 2,
 1,
 1,
 4,
 1,
 0,
 2,
 0,
 3,
 3,
 0,
 3,
 0,
 3,
 1,
 0,
 4,
 1,
 1,
 3,
 0,
 2,
 1,
 0,
 3,
 3,
 1,
 0,
 0,
 2,
 1,
 1,
 1,
 0,
 1,
 3,
 1,
 0,
 0,
 3,
 3,
 2,
 0,
 3,
 4,
 3,
 1,
 4,
 2,
 3,
 2,
 0,
 2,
 3,
 4,
 2,


In [ ]:
df['gpt-label'] = labels

In [ ]:
df

text  label  gpt-label  \
0    Sustainable strategy ‘red lines’ For our susta...      2          2   
1    Verizon’s environmental, health and safety man...      3          3   
2    In 2019, the Company closed a series of transa...      2          0   
3    In December 2020, the AUC approved the Electri...      2          0   
4    Finally, there is a reputational risk linked t...      2          3   
..                                                 ...    ...        ...   
395  In 2020, Banco do Brasil Foundation celebrated...      2          2   
396  Climate change is producing changes in weather...      2          3   
397  A sound and certain regulatory and fiscal envi...      0          0   
398  Across our global workforce, 20% of Gold Field...      0          1   
399  The purchase price of these raw materials and ...      0          0   

                                      gpt-explanations  \
0    the provided text is about strategy for sustai...   
1    the provided text is about risk for sustainabi...   
2    the provided text is not about sustainability,...   
3    the provided text is not about sustainability,...   
4    the provided text is about risk for sustainabi...   
..                                                 ...   
395  the provided text is about strategy for sustai...   
396  the provided text is about risk for sustainabi...   
397  the provided text is not about sustainability,...   
398  the provided text is about metrics for sustain...   
399  the provided text is not about sustainability,...   

                                                points  
0    Points:\n1. Sustainable strategy range\n2. Pro...  
1    Points:\n1. Verizon has an environmental, heal...  
2    Points:\n- The Company sold its Canadian fossi...  
3    Points:\n- AUC approved deferral of distributi...  
4    - Reputational risk for oil companies \n- Poss...  
..                                                 ...  
395  Points:\n- Banco do Brasil Foundation celebrat...  
396  1. Climate change leads to changes in environm...  
397  Points:\n1. Regulatory and fiscal environment ...  
398  - Gold Fields' global workforce has 20% women ...  
399  Key Points: \n\n- Raw material and fuel prices...  

[400 rows x 5 columns]

In [ ]:
df['gpt-label'].value_counts()

3    123
1    113
0     87
2     53
4     24
Name: gpt-label, dtype: int64

In [ ]:
df.to_csv("/content/drive/MyDrive/DS-Environment-Project/ChatGPT Results/scikit-llm/chatgpt_climate_tcfd_recommendations_scikit_llm.csv",index=False)

In [ ]:
# calculate the precision and f1 score for df columns label and prediction
from sklearn.metrics import precision_recall_fscore_support,f1_score
sent_col = 'gpt-label'
print(precision_recall_fscore_support(df['label'], df[sent_col], average='macro'))

# f1 score only
print(f1_score(df['label'], df[sent_col], average='macro'))

# confusion matrix
from sklearn.metrics import confusion_matrix
print(confusion_matrix(df['label'], df[sent_col]))

# performnce report

from sklearn.metrics import classification_report
print(classification_report(df['label'], df[sent_col]))

(0.45759588424961695, 0.5132752804362666, 0.4198191064265232, None)
0.4198191064265232
[[51  2  2 21  4]
 [ 5 38  5  1  0]
 [30 56 40 63  8]
 [ 0 14  2 29  3]
 [ 1  3  4  9  9]]
              precision    recall  f1-score   support

           0       0.59      0.64      0.61        80
           1       0.34      0.78      0.47        49
           2       0.75      0.20      0.32       197
           3       0.24      0.60      0.34        48
           4       0.38      0.35      0.36        26

    accuracy                           0.42       400
   macro avg       0.46      0.51      0.42       400
weighted avg       0.58      0.42      0.40       400



# testing Climate Specificity on ChatGPT by using summarization of most important points in the paragraphs
## Classification classes:
### 0 - non-specific
### 1 - specific

In [1]:
!pip install datasets
from datasets import load_dataset

dataset = load_dataset("climatebert/climate_specificity")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 42.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 19.2 MB/s eta 0:00:00


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/320 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/climatebert___parquet/climatebert--climate_specificity-cc194c8c8abc022c/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 320
    })
})

In [6]:
dataset['test']

Dataset({
    features: ['text', 'label'],
    num_rows: 320
})

In [7]:
data = []
for i in range(0,len(dataset['test']['text'])):
  data.append([dataset['test']['text'][i],dataset['test']['label'][i]])

print(data)

[['Sustainable strategy ‘red lines’ For our sustainable strategy range, we incorporate a series of proprietary ‘red lines’ in order to ensure the poorest- performing companies from an ESG perspective are not eligible for investment.', 0], ['Verizon’s environmental, health and safety management system provides a framework for identifying, controlling, and reducing the risks associated with the environments in which we operate. Besides regular management system assessments, internal and third-party compliance audits and inspections are performed annually at hundreds of facilities worldwide. The goal of these assessments is to identify and correct site-specific issues, and to educate and empower facility managers and supervisors to implement corrective actions. Verizon’s environment, health and safety efforts are directed and supported by experienced experts around the world that support our operations and facilities.', 0], ['In 2019, the Company closed a series of transactions related to

In [8]:
import pandas as pd
df = pd.DataFrame(data=data,columns=["text","label"])

In [9]:
df

text  label
0    Sustainable strategy ‘red lines’ For our susta...      0
1    Verizon’s environmental, health and safety man...      0
2    In 2019, the Company closed a series of transa...      1
3    In December 2020, the AUC approved the Electri...      1
4    Finally, there is a reputational risk linked t...      0
..                                                 ...    ...
315  Indirect emissions result from operational act...      0
316  All data in this TCFD report is as of, or for ...      0
317  Outcome: The bank explained that it would be w...      1
318  In 2020, Banco do Brasil Foundation celebrated...      0
319  Climate change is producing changes in weather...      1

[320 rows x 2 columns]

In [10]:
prompt = 'You are the sustainability, environment, and climate change expert. Read the following paragraph and extract the most important points from the text and return only the points and their explanations: \n\n'
chain_prompt = 'Read the following points and answer only with one number that is the overall specificity of all points summarized without any explanations. Answer only with 0 if the points are not specific about sustainabilty, environment and climate change, and answer only with 1 if the points are specific about sustainabilty, environment and climate change: \n\n'
texts = df["text"].to_list()
rez, rez_keys, summarized_points = batch_gpt_resp_as_part_prompt(prompt,chain_prompt, texts, 1)

1 431 - 1 1: 2 889 - 1 1: 3 818 - 1 1: 4 1131 - 0 0: 5 621 - 1 1: 6 590 - 1 1: 7 587 - 1 1: 8 731 - 1 1: 9 506 - 1 1: 10 462 - 1 1: 
11 474 - 1 1: 12 428 - 1 1: 13 487 - 0 0: 14 648 - 1 1: 15 571 - 1 1: 16 402 - 1 1: 17 878 - 0 0: 18 637 - 0 0: 19 913 - 1 1: 20 858 - 1 1: 
21 813 - 0 0: 22 564 - 1 1: 23 513 - 1 1: 24 453 - 1 1: 25 684 - 1 1: 26 816 - 1 1: error calling API, retrying...
error parsingKey points:
- Strains on the natural environment intensifying due to climate change and water scarcity
- Global momentum to tackle climate change following the Paris Agreement, aiming to limit temperature rise below 2 degrees Celsius above pre-industrial levels
- Concerns about the planet and society matched by concerns about health, including obesity and insufficient nourishment 
- Opportunities to meet growing consumer needs in the food industry.
27 814 - Key points:
- Strains on the natural environment intensifying due to climate change and water scarcity
- Global momentum to tackle clima

In [11]:
rez

['1',
 '1',
 '1',
 '0',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '0',
 '1',
 '1',
 '1',
 '0',
 '0',
 '1',
 '1',
 '0',
 '1',
 '1',
 '1',
 '1',
 '1',
 'Key points:\n- Strains on the natural environment intensifying due to climate change and water scarcity\n- Global momentum to tackle climate change following the Paris Agreement, aiming to limit temperature rise below 2 degrees Celsius above pre-industrial levels\n- Concerns about the planet and society matched by concerns about health, including obesity and insufficient nourishment \n- Opportunities to meet growing consumer needs in the food industry.',
 '1',
 '0',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '0',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '0',
 '1',
 '1',
 '1',
 '0',
 '0',
 '1.',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1.',
 '1',
 '1',
 '1',
 '0',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '0',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '0',
 '1

In [12]:
rez_keys

[1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 -1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 -1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1

In [13]:
df['gpt-label'] = rez_keys

In [14]:
df['gpt-explanations'] = rez

In [15]:
df['gpt-summarized'] = summarized_points

In [16]:
df

text  label  gpt-label  \
0    Sustainable strategy ‘red lines’ For our susta...      0          1   
1    Verizon’s environmental, health and safety man...      0          1   
2    In 2019, the Company closed a series of transa...      1          1   
3    In December 2020, the AUC approved the Electri...      1          0   
4    Finally, there is a reputational risk linked t...      0          1   
..                                                 ...    ...        ...   
315  Indirect emissions result from operational act...      0          1   
316  All data in this TCFD report is as of, or for ...      0          0   
317  Outcome: The bank explained that it would be w...      1          2   
318  In 2020, Banco do Brasil Foundation celebrated...      0          1   
319  Climate change is producing changes in weather...      1          1   

    gpt-explanations                                     gpt-summarized  
0                  1  Points: \n1. Sustainable strategy range \n2. P...  
1                  1  1. Verizon has an environmental, health, and s...  
2                  1  Points:\n\n1. The Company sold its Canadian fo...  
3                  0  Points:\n1. AUC approved deferral of distribut...  
4                  1  Key Points:\n- Reputational risk linked to oil...  
..               ...                                                ...  
315                1  1. Indirect emissions result from operational ...  
316                0  1. TCFD report data is for the year-ended Dece...  
317                2  Points:\n1. Bank is winding down its fossil fu...  
318                1  Points:\n- Banco do Brasil Foundation celebrat...  
319                1  Points:\n1. Climate change is changing weather...  

[320 rows x 5 columns]

In [17]:
df['gpt-label'].value_counts()

 1    259
 0     55
-1      5
 2      1
Name: gpt-label, dtype: int64

In [18]:
df[df['gpt-label']==-1]

text  label  gpt-label  \
26   Strains on the natural environment are intensi...      0         -1   
195  While we seek to partner with organizations th...      1         -1   
278  The objectives of the CCWG, which reflect the ...      1         -1   
306  Adverse weather, such as hurricanes and tropic...      1         -1   
313  Reduction targets We continue to set reduction...      0         -1   

                                      gpt-explanations  \
26   Key points:\n- Strains on the natural environm...   
195  Points:\n\n- Partner with organizations that m...   
278  Points:\n- CCWG objectives reflect risks and o...   
306  Points:\n1. Adverse weather can affect Offshor...   
313  - Reduction targets set to limit impact on env...   

                                        gpt-summarized  
26   Key points:\n- Strains on the natural environm...  
195  Points:\n\n- Partner with organizations that m...  
278  Points:\n- CCWG objectives reflect risks and o...  
306  Points:\n1. Adverse weather can affect Offshor...  
313  - Reduction targets set to limit impact on env...

In [19]:
df['gpt-label'] = df['gpt-label'].replace(-1,1)

In [20]:
df[df['gpt-label']==2]

text  label  gpt-label  \
317  Outcome: The bank explained that it would be w...      1          2   

    gpt-explanations                                     gpt-summarized  
317                2  Points:\n1. Bank is winding down its fossil fu...

In [21]:
df['gpt-label'] = df['gpt-label'].replace(2,1)

In [22]:
df['gpt-label'].value_counts()

1    265
0     55
Name: gpt-label, dtype: int64

In [23]:
df.to_csv("/content/drive/MyDrive/DS-Environment-Project/ChatGPT Results/chatgpt_climate_specificity_chained_prompts.csv",index=False)

In [24]:
# calculate the precision and f1 score for df columns label and prediction
from sklearn.metrics import precision_recall_fscore_support,f1_score
sent_col = 'gpt-label'
print(precision_recall_fscore_support(df['label'], df[sent_col], average='macro'))

# f1 score only
print(f1_score(df['label'], df[sent_col], average='macro'))

# confusion matrix
from sklearn.metrics import confusion_matrix
print(confusion_matrix(df['label'], df[sent_col]))

# performnce report

from sklearn.metrics import classification_report
print(classification_report(df['label'], df[sent_col]))

(0.44665523156089193, 0.46791300400313646, 0.36507936507936506, None)
0.36507936507936506
[[ 29 168]
 [ 26  97]]
              precision    recall  f1-score   support

           0       0.53      0.15      0.23       197
           1       0.37      0.79      0.50       123

    accuracy                           0.39       320
   macro avg       0.45      0.47      0.37       320
weighted avg       0.47      0.39      0.33       320



## adding the summarized points to scikit-llm library

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/DS-Environment-Project/ChatGPT Results/chatgpt_climate_tcfd_chained_prompts.csv")

In [ ]:
df_sent = pd.read_csv("/content/drive/MyDrive/IntroToDataScience/DS-Environment-Project/ChatGPT Results/chatgpt_climate_sentiment.csv")

In [ ]:
df

text  label  gpt-label  \
0    Sustainable strategy ‘red lines’ For our susta...      2          2   
1    Verizon’s environmental, health and safety man...      3          3   
2    In 2019, the Company closed a series of transa...      2          0   
3    In December 2020, the AUC approved the Electri...      2          4   
4    Finally, there is a reputational risk linked t...      2          3   
..                                                 ...    ...        ...   
395  In 2020, Banco do Brasil Foundation celebrated...      2          1   
396  Climate change is producing changes in weather...      2          1   
397  A sound and certain regulatory and fiscal envi...      0          3   
398  Across our global workforce, 20% of Gold Field...      0          1   
399  The purchase price of these raw materials and ...      0          0   

                            gpt-explanations  \
0                                          2   
1                                          3   
2                                          0   
3                                          4   
4                                          3   
..                                       ...   
395                                        1   
396                                        1   
397  to investment \n0 (not climate-related)   
398                                        1   
399                                        0   

                                                points  
0    Points:\n1. Sustainable strategy range\n2. Pro...  
1    Points:\n1. Verizon has an environmental, heal...  
2    Points:\n- The Company sold its Canadian fossi...  
3    Points:\n- AUC approved deferral of distributi...  
4    - Reputational risk for oil companies \n- Poss...  
..                                                 ...  
395  Points:\n- Banco do Brasil Foundation celebrat...  
396  1. Climate change leads to changes in environm...  
397  Points:\n1. Regulatory and fiscal environment ...  
398  - Gold Fields' global workforce has 20% women ...  
399  Key Points: \n\n- Raw material and fuel prices...  

[400 rows x 5 columns]

In [ ]:
df['label'] = df_sent['label']

In [ ]:
df = df[:320]

In [ ]:
df['label'] = df_sent['label']

<ipython-input-83-b5fc60e5602d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = df_sent['label']


In [ ]:
df

text  label  gpt-label  \
0    Sustainable strategy ‘red lines’ For our susta...      0          2   
1    Verizon’s environmental, health and safety man...      1          2   
2    In 2019, the Company closed a series of transa...      1          2   
3    In December 2020, the AUC approved the Electri...      0          1   
4    Finally, there is a reputational risk linked t...      0          0   
..                                                 ...    ...        ...   
315  Energy Infrastructure revenues of $195 million...      1          1   
316  Financial planning We acknowledge that there a...      1          2   
317  : PCG Group has adopted the PETRONAS Whistlebl...      1          1   
318  Stakeholder dialogue and transparency ⌐ Engagi...      2          2   
319  Foreign, federal, state and local regulatory a...      0          0   

                                      gpt-explanations  \
0    the sentiment of the provided text is opportun...   
1    the sentiment of the provided text is opportun...   
2    the sentiment of the provided text is opportun...   
3    the sentiment of the provided text is neutral ...   
4    the sentiment of the provided text is risk abo...   
..                                                 ...   
315  the sentiment of the provided text is neutral ...   
316  the sentiment of the provided text is opportun...   
317  the sentiment of the provided text is neutral ...   
318  the sentiment of the provided text is opportun...   
319  the sentiment of the provided text is risk abo...   

                                                points  
0    Points:\n1. Sustainable strategy range\n2. Pro...  
1    Points:\n1. Verizon has an environmental, heal...  
2    Points:\n- The Company sold its Canadian fossi...  
3    Points:\n- AUC approved deferral of distributi...  
4    - Reputational risk for oil companies \n- Poss...  
..                                                 ...  
315  - Energy Infrastructure revenues in 2020 were ...  
316  Points:\n-Company intends to integrate climate...  
317  Points:\n- PCG Group has adopted the PETRONAS ...  
318  - Adidas engages openly with stakeholders and ...  
319  Points:\n\n1. Proposed legislation and regulat...  

[320 rows x 5 columns]

In [ ]:
from skllm import ZeroShotGPTClassifier

X = df['points']
Y = df['label']


clf = ZeroShotGPTClassifier()
clf.fit(None, ["the sentiment of the provided text is risk about sustainability, environment and climate change","the sentiment of the provided text is neutral about sustainability, environment and climate change","the sentiment of the provided text is opportunity about sustainability, environment and climate change"])
preds = clf.predict(X)

100%|██████████| 320/320 [07:58<00:00,  1.49s/it]


In [ ]:
preds

['the sentiment of the provided text is opportunity about sustainability, environment and climate change',
 'the sentiment of the provided text is opportunity about sustainability, environment and climate change',
 'the sentiment of the provided text is opportunity about sustainability, environment and climate change',
 'the sentiment of the provided text is neutral about sustainability, environment and climate change',
 'the sentiment of the provided text is risk about sustainability, environment and climate change',
 'the sentiment of the provided text is opportunity about sustainability, environment and climate change',
 'the sentiment of the provided text is risk about sustainability, environment and climate change',
 'the sentiment of the provided text is opportunity about sustainability, environment and climate change',
 'the sentiment of the provided text is opportunity about sustainability, environment and climate change',
 'the sentiment of the provided text is opportunity abo

In [ ]:
df['gpt-explanations'] = preds

<ipython-input-87-263adef27334>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['gpt-explanations'] = preds


In [ ]:
df[df['gpt-explanations'] == "the sentiment of the provided text is risk about sustainability, environment and climate change"]

text  label  gpt-label  \
4    Finally, there is a reputational risk linked t...      0          0   
6    The Group and its customers are exposed to cli...      0          0   
11   Climate-related risks and opportunities have g...      2          0   
13   An emerging risk is defined as a risk which ha...      1          0   
17   We emit greenhouse gases both directly and ind...      1          0   
..                                                 ...    ...        ...   
302  Nedbank has a Damage to Physical Assets scenar...      1          0   
310  Rising public concern related to climate chang...      1          0   
311  In addition, certain environmental laws assess...      1          0   
312  However, we believe that the data currently di...      1          0   
319  Foreign, federal, state and local regulatory a...      0          0   

                                      gpt-explanations  \
4    the sentiment of the provided text is risk abo...   
6    the sentiment of the provided text is risk abo...   
11   the sentiment of the provided text is risk abo...   
13   the sentiment of the provided text is risk abo...   
17   the sentiment of the provided text is risk abo...   
..                                                 ...   
302  the sentiment of the provided text is risk abo...   
310  the sentiment of the provided text is risk abo...   
311  the sentiment of the provided text is risk abo...   
312  the sentiment of the provided text is risk abo...   
319  the sentiment of the provided text is risk abo...   

                                                points  
4    - Reputational risk for oil companies \n- Poss...  
6    1. The Group and its customers are exposed to ...  
11   Points:\n\n1. Climate-related risks and opport...  
13   Points:\n\n1. Emerging risk is defined as a ri...  
17   Points:\n\n- Greenhouse gases are emitted both...  
..                                                 ...  
302  Points:\n\n- Nedbank has a scenario for physic...  
310  Points:\n1. Rising public concern related to c...  
311  1. Environmental laws can hold current or prev...  
312  Points: \n- Data disclosed by companies does n...  
319  Points:\n\n1. Proposed legislation and regulat...  

[74 rows x 5 columns]

In [ ]:
labels = []

for label in preds:
  if label == "the sentiment of the provided text is risk about sustainability, environment and climate change":
    labels.append(0)
  elif label == "the sentiment of the provided text is neutral about sustainability, environment and climate change":
    labels.append(1)
  elif label == "the sentiment of the provided text is opportunity about sustainability, environment and climate change":
    labels.append(2)


labels

[2,
 2,
 2,
 1,
 0,
 2,
 0,
 2,
 2,
 2,
 1,
 0,
 1,
 0,
 2,
 2,
 2,
 0,
 2,
 1,
 1,
 2,
 2,
 0,
 0,
 0,
 2,
 2,
 2,
 0,
 0,
 2,
 2,
 0,
 1,
 1,
 2,
 2,
 0,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 1,
 2,
 2,
 1,
 2,
 0,
 2,
 2,
 1,
 1,
 1,
 2,
 2,
 2,
 0,
 0,
 2,
 2,
 1,
 2,
 2,
 2,
 0,
 1,
 0,
 2,
 0,
 0,
 2,
 2,
 0,
 1,
 0,
 2,
 1,
 2,
 1,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 0,
 0,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 1,
 2,
 2,
 0,
 2,
 2,
 0,
 1,
 1,
 0,
 2,
 2,
 0,
 2,
 0,
 1,
 1,
 2,
 2,
 1,
 2,
 2,
 1,
 2,
 1,
 0,
 0,
 1,
 2,
 2,
 2,
 1,
 2,
 0,
 0,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 0,
 2,
 1,
 1,
 2,
 0,
 0,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 0,
 0,
 2,
 1,
 2,
 2,
 2,
 2,
 1,
 2,
 0,
 0,
 1,
 1,
 1,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 1,
 1,
 2,
 0,
 2,
 2,
 2,
 0,
 2,
 2,
 1,
 2,
 1,
 0,
 2,
 1,
 0,
 0,
 0,
 2,
 1,
 2,
 0,
 1,
 1,
 0,
 2,
 2,
 1,
 2,
 1,
 2,
 1,
 1,
 2,
 2,
 2,
 2,
 1,
 2,
 2,
 1,
 1,
 2,
 0,
 0,
 2,
 1,
 0,
 1,
 2,
 2,
 2,
 2,
 0,
 2,
 1,
 2,
 1,
 2,
 2,


In [ ]:
df['gpt-label'] = labels

<ipython-input-90-bd7b018092de>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['gpt-label'] = labels


In [ ]:
df

text  label  gpt-label  \
0    Sustainable strategy ‘red lines’ For our susta...      0          2   
1    Verizon’s environmental, health and safety man...      1          2   
2    In 2019, the Company closed a series of transa...      1          2   
3    In December 2020, the AUC approved the Electri...      0          1   
4    Finally, there is a reputational risk linked t...      0          0   
..                                                 ...    ...        ...   
315  Energy Infrastructure revenues of $195 million...      1          1   
316  Financial planning We acknowledge that there a...      1          2   
317  : PCG Group has adopted the PETRONAS Whistlebl...      1          1   
318  Stakeholder dialogue and transparency ⌐ Engagi...      2          2   
319  Foreign, federal, state and local regulatory a...      0          0   

                                      gpt-explanations  \
0    the sentiment of the provided text is opportun...   
1    the sentiment of the provided text is opportun...   
2    the sentiment of the provided text is opportun...   
3    the sentiment of the provided text is neutral ...   
4    the sentiment of the provided text is risk abo...   
..                                                 ...   
315  the sentiment of the provided text is neutral ...   
316  the sentiment of the provided text is opportun...   
317  the sentiment of the provided text is neutral ...   
318  the sentiment of the provided text is opportun...   
319  the sentiment of the provided text is risk abo...   

                                                points  
0    Points:\n1. Sustainable strategy range\n2. Pro...  
1    Points:\n1. Verizon has an environmental, heal...  
2    Points:\n- The Company sold its Canadian fossi...  
3    Points:\n- AUC approved deferral of distributi...  
4    - Reputational risk for oil companies \n- Poss...  
..                                                 ...  
315  - Energy Infrastructure revenues in 2020 were ...  
316  Points:\n-Company intends to integrate climate...  
317  Points:\n- PCG Group has adopted the PETRONAS ...  
318  - Adidas engages openly with stakeholders and ...  
319  Points:\n\n1. Proposed legislation and regulat...  

[320 rows x 5 columns]

In [ ]:
df['gpt-label'].value_counts()

2    179
0     74
1     67
Name: gpt-label, dtype: int64

In [ ]:
df.to_csv("/content/drive/MyDrive/DS-Environment-Project/ChatGPT Results/scikit-llm/chatgpt_climate_sentiment_chained_prompt.csv",index=False)

In [ ]:
# calculate the precision and f1 score for df columns label and prediction
from sklearn.metrics import precision_recall_fscore_support,f1_score
sent_col = 'gpt-label'
print(precision_recall_fscore_support(df['label'], df[sent_col], average='macro'))

# f1 score only
print(f1_score(df['label'], df[sent_col], average='macro'))

# confusion matrix
from sklearn.metrics import confusion_matrix
print(confusion_matrix(df['label'], df[sent_col]))

# performnce report

from sklearn.metrics import classification_report
print(classification_report(df['label'], df[sent_col]))

(0.3103037582734072, 0.2989660810112524, 0.24879227053140096, None)
0.24879227053140096
[[21 21 64]
 [39 34 90]
 [14 12 25]]
              precision    recall  f1-score   support

           0       0.28      0.20      0.23       106
           1       0.51      0.21      0.30       163
           2       0.14      0.49      0.22        51

    accuracy                           0.25       320
   macro avg       0.31      0.30      0.25       320
weighted avg       0.37      0.25      0.26       320



# testing Climate Commitments and Actions on ChatGPT by using summarization of most important points in the paragraphs
## Classification classes:
### 0 - is not about climate commitments and actions
### 1 - is about climate commitments and actions

In [25]:
!pip install datasets
from datasets import load_dataset

dataset = load_dataset("climatebert/climate_commitments_actions")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/320 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/climatebert___parquet/climatebert--climate_commitments_actions-c39067b6628a5441/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [26]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 320
    })
})

In [27]:
dataset['test']

Dataset({
    features: ['text', 'label'],
    num_rows: 320
})

In [28]:
data = []
for i in range(0,len(dataset['test']['text'])):
  data.append([dataset['test']['text'][i],dataset['test']['label'][i]])

print(data)

[['Sustainable strategy ‘red lines’ For our sustainable strategy range, we incorporate a series of proprietary ‘red lines’ in order to ensure the poorest- performing companies from an ESG perspective are not eligible for investment.', 1], ['Verizon’s environmental, health and safety management system provides a framework for identifying, controlling, and reducing the risks associated with the environments in which we operate. Besides regular management system assessments, internal and third-party compliance audits and inspections are performed annually at hundreds of facilities worldwide. The goal of these assessments is to identify and correct site-specific issues, and to educate and empower facility managers and supervisors to implement corrective actions. Verizon’s environment, health and safety efforts are directed and supported by experienced experts around the world that support our operations and facilities.', 0], ['In 2019, the Company closed a series of transactions related to

In [29]:
import pandas as pd
df = pd.DataFrame(data=data,columns=["text","label"])

In [30]:
df

text  label
0    Sustainable strategy ‘red lines’ For our susta...      1
1    Verizon’s environmental, health and safety man...      0
2    In 2019, the Company closed a series of transa...      1
3    In December 2020, the AUC approved the Electri...      0
4    Finally, there is a reputational risk linked t...      0
..                                                 ...    ...
315  Indirect emissions result from operational act...      0
316  All data in this TCFD report is as of, or for ...      0
317  Outcome: The bank explained that it would be w...      1
318  In 2020, Banco do Brasil Foundation celebrated...      1
319  Climate change is producing changes in weather...      0

[320 rows x 2 columns]

In [31]:
prompt = 'You are the sustainability, environment, and climate change expert. Read the following paragraph and extract the most important points from the text and return only the points and their explanations: \n\n'
chain_prompt = 'Read the following points and answer only with one number that is the overall class of all points summarized without any explanations. Answer only with 0 if the points are not about climate commitments and actions, and answer only with 1 if the points are about climate commitments and actions: \n\n'
texts = df["text"].to_list()
rez, rez_keys, summarized_points = batch_gpt_resp_as_part_prompt(prompt,chain_prompt, texts, 1)

1 431 - 1 1: 2 889 - 0 0: 3 818 - 0 0: 4 1131 - 0 0: 5 621 - 1 1: 6 590 - 1 1: 7 587 - 0 0: 8 731 - 1 1: 9 506 - 1 1: 10 462 - 1 1: 
11 474 - 1 1: 12 428 - 1 1: 13 487 - 0 0: 14 648 - 1 1: 15 571 - 1 1: 16 402 - 1 1: 17 878 - 0 0: 18 637 - 0 0: 19 913 - 1 1: 20 858 - 1 1: 
21 813 - 0 0: 22 564 - 1 1: 23 513 - 1 1: 24 453 - 1 1: 25 684 - 1 1: 26 816 - 1 1: 27 814 - 1 1: 28 411 - 1 1: 29 510 - 1 1: 30 705 - 1 1: 
31 869 - 1 1: 32 1039 - 1 1: 33 794 - 1. 1: 34 536 - 1 1: 35 709 - 1 1: 36 526 - 1 1: 37 626 - 1 1: 38 504 - 1 1: 39 781 - 1 1: 40 660 - 1 1: 
41 452 - 1 1: 42 499 - 1 1: 43 759 - 1 1: 44 756 - 1 1: 45 470 - 0 0: 46 431 - 1 1: 47 420 - 1 1: 48 657 - 1 1: 49 526 - 1 1: error calling API, retrying...
error parsing- Sustainability issues in the tire industry include sustainable natural rubber and microplastics.
- The sustainability of the natural rubber supply chain is associated with human rights, environmental protection, transparent management, productivity, quality and quality 

In [32]:
rez

['1',
 '0',
 '0',
 '0',
 '1',
 '1',
 '0',
 '1',
 '1',
 '1',
 '1',
 '1',
 '0',
 '1',
 '1',
 '1',
 '0',
 '0',
 '1',
 '1',
 '0',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1.',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '0',
 '1',
 '1',
 '1',
 '1',
 "- Sustainability issues in the tire industry include sustainable natural rubber and microplastics.\n- The sustainability of the natural rubber supply chain is associated with human rights, environmental protection, transparent management, productivity, quality and quality of life.\n- Hankook Tire & Technology is addressing these issues through the Global Platform for Sustainable Natural Rubber GPSNR.\n- Hankook Tire & Technology is also researching microplastics for tire wear particles.\n- LG Chem is a major supplier of Hankook Tire & Technology and its sustainability efforts will impact the company's responsible sourcing policy and products.\n- Collaboration among companies or industries is ne

In [33]:
rez_keys

[1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 -1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 -1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1

In [34]:
df['gpt-label'] = rez_keys

In [35]:
df['gpt-explanations'] = rez

In [36]:
df['gpt-summarized'] = summarized_points

In [37]:
df

text  label  gpt-label  \
0    Sustainable strategy ‘red lines’ For our susta...      1          1   
1    Verizon’s environmental, health and safety man...      0          0   
2    In 2019, the Company closed a series of transa...      1          0   
3    In December 2020, the AUC approved the Electri...      0          0   
4    Finally, there is a reputational risk linked t...      0          1   
..                                                 ...    ...        ...   
315  Indirect emissions result from operational act...      0          1   
316  All data in this TCFD report is as of, or for ...      0          0   
317  Outcome: The bank explained that it would be w...      1          1   
318  In 2020, Banco do Brasil Foundation celebrated...      1          0   
319  Climate change is producing changes in weather...      0          1   

    gpt-explanations                                     gpt-summarized  
0                  1  Points: \n1. Sustainable strategy range \n2. P...  
1                  0  - Verizon has an environmental, health, and sa...  
2                  0  Points:\n1. The Company sold its Canadian foss...  
3                  0  Points:\n1. AUC approved deferral of compulsor...  
4                  1  Points: \n1. Reputational risk for oil compani...  
..               ...                                                ...  
315                1  1. Indirect emissions come from activities not...  
316                0  1. TCFD report data is up until December 31, 2...  
317                1  Points: \n1. The bank is winding down its foss...  
318                0  Points: \n1. Banco do Brasil Foundation celebr...  
319                1  -Climate change is causing changes in weather ...  

[320 rows x 5 columns]

In [38]:
df['gpt-label'].value_counts()

 1    243
 0     73
-1      4
Name: gpt-label, dtype: int64

In [39]:
df[df['gpt-label']==-1]

text  label  gpt-label  \
49   What are the latest sustainability issues in y...      1         -1   
159  Innovation and Digital Two enablers will set u...      0         -1   
292  In addition to capital investments in the Regu...      1         -1   
294  We also anticipate that the potential effects ...      0         -1   

                                      gpt-explanations  \
49   - Sustainability issues in the tire industry i...   
159  - Two enablers for success: deep and broad inn...   
292  - Canadian Utilities plans to invest in:\n\n1....   
294  - Climate change effects will impact operation...   

                                        gpt-summarized  
49   - Sustainability issues in the tire industry i...  
159  - Two enablers for success: deep and broad inn...  
292  - Canadian Utilities plans to invest in:\n\n1....  
294  - Climate change effects will impact operation...

In [40]:
df['gpt-label'] = df['gpt-label'].replace(-1,1)

In [41]:
df['gpt-label'].value_counts()

1    247
0     73
Name: gpt-label, dtype: int64

In [42]:
df.to_csv("/content/drive/MyDrive/DS-Environment-Project/ChatGPT Results/chatgpt_climate_commitments_and_actions_chained_prompts.csv",index=False)

In [43]:
# calculate the precision and f1 score for df columns label and prediction
from sklearn.metrics import precision_recall_fscore_support,f1_score
sent_col = 'gpt-label'
print(precision_recall_fscore_support(df['label'], df[sent_col], average='macro'))

# f1 score only
print(f1_score(df['label'], df[sent_col], average='macro'))

# confusion matrix
from sklearn.metrics import confusion_matrix
print(confusion_matrix(df['label'], df[sent_col]))

# performnce report

from sklearn.metrics import classification_report
print(classification_report(df['label'], df[sent_col]))

(0.53865564860518, 0.5320371391799963, 0.41832473593711617, None)
0.41832473593711617
[[ 55 167]
 [ 18  80]]
              precision    recall  f1-score   support

           0       0.75      0.25      0.37       222
           1       0.32      0.82      0.46        98

    accuracy                           0.42       320
   macro avg       0.54      0.53      0.42       320
weighted avg       0.62      0.42      0.40       320



## adding the summarized points to scikit-llm library

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/DS-Environment-Project/ChatGPT Results/chatgpt_climate_tcfd_chained_prompts.csv")

In [ ]:
df_sent = pd.read_csv("/content/drive/MyDrive/IntroToDataScience/DS-Environment-Project/ChatGPT Results/chatgpt_climate_sentiment.csv")

In [ ]:
df

text  label  gpt-label  \
0    Sustainable strategy ‘red lines’ For our susta...      2          2   
1    Verizon’s environmental, health and safety man...      3          3   
2    In 2019, the Company closed a series of transa...      2          0   
3    In December 2020, the AUC approved the Electri...      2          4   
4    Finally, there is a reputational risk linked t...      2          3   
..                                                 ...    ...        ...   
395  In 2020, Banco do Brasil Foundation celebrated...      2          1   
396  Climate change is producing changes in weather...      2          1   
397  A sound and certain regulatory and fiscal envi...      0          3   
398  Across our global workforce, 20% of Gold Field...      0          1   
399  The purchase price of these raw materials and ...      0          0   

                            gpt-explanations  \
0                                          2   
1                                          3   
2                                          0   
3                                          4   
4                                          3   
..                                       ...   
395                                        1   
396                                        1   
397  to investment \n0 (not climate-related)   
398                                        1   
399                                        0   

                                                points  
0    Points:\n1. Sustainable strategy range\n2. Pro...  
1    Points:\n1. Verizon has an environmental, heal...  
2    Points:\n- The Company sold its Canadian fossi...  
3    Points:\n- AUC approved deferral of distributi...  
4    - Reputational risk for oil companies \n- Poss...  
..                                                 ...  
395  Points:\n- Banco do Brasil Foundation celebrat...  
396  1. Climate change leads to changes in environm...  
397  Points:\n1. Regulatory and fiscal environment ...  
398  - Gold Fields' global workforce has 20% women ...  
399  Key Points: \n\n- Raw material and fuel prices...  

[400 rows x 5 columns]

In [ ]:
df['label'] = df_sent['label']

In [ ]:
df = df[:320]

In [ ]:
df['label'] = df_sent['label']

<ipython-input-83-b5fc60e5602d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = df_sent['label']


In [ ]:
df

text  label  gpt-label  \
0    Sustainable strategy ‘red lines’ For our susta...      0          2   
1    Verizon’s environmental, health and safety man...      1          2   
2    In 2019, the Company closed a series of transa...      1          2   
3    In December 2020, the AUC approved the Electri...      0          1   
4    Finally, there is a reputational risk linked t...      0          0   
..                                                 ...    ...        ...   
315  Energy Infrastructure revenues of $195 million...      1          1   
316  Financial planning We acknowledge that there a...      1          2   
317  : PCG Group has adopted the PETRONAS Whistlebl...      1          1   
318  Stakeholder dialogue and transparency ⌐ Engagi...      2          2   
319  Foreign, federal, state and local regulatory a...      0          0   

                                      gpt-explanations  \
0    the sentiment of the provided text is opportun...   
1    the sentiment of the provided text is opportun...   
2    the sentiment of the provided text is opportun...   
3    the sentiment of the provided text is neutral ...   
4    the sentiment of the provided text is risk abo...   
..                                                 ...   
315  the sentiment of the provided text is neutral ...   
316  the sentiment of the provided text is opportun...   
317  the sentiment of the provided text is neutral ...   
318  the sentiment of the provided text is opportun...   
319  the sentiment of the provided text is risk abo...   

                                                points  
0    Points:\n1. Sustainable strategy range\n2. Pro...  
1    Points:\n1. Verizon has an environmental, heal...  
2    Points:\n- The Company sold its Canadian fossi...  
3    Points:\n- AUC approved deferral of distributi...  
4    - Reputational risk for oil companies \n- Poss...  
..                                                 ...  
315  - Energy Infrastructure revenues in 2020 were ...  
316  Points:\n-Company intends to integrate climate...  
317  Points:\n- PCG Group has adopted the PETRONAS ...  
318  - Adidas engages openly with stakeholders and ...  
319  Points:\n\n1. Proposed legislation and regulat...  

[320 rows x 5 columns]

In [ ]:
from skllm import ZeroShotGPTClassifier

X = df['points']
Y = df['label']


clf = ZeroShotGPTClassifier()
clf.fit(None, ["the sentiment of the provided text is risk about sustainability, environment and climate change","the sentiment of the provided text is neutral about sustainability, environment and climate change","the sentiment of the provided text is opportunity about sustainability, environment and climate change"])
preds = clf.predict(X)

100%|██████████| 320/320 [07:58<00:00,  1.49s/it]


In [ ]:
preds

['the sentiment of the provided text is opportunity about sustainability, environment and climate change',
 'the sentiment of the provided text is opportunity about sustainability, environment and climate change',
 'the sentiment of the provided text is opportunity about sustainability, environment and climate change',
 'the sentiment of the provided text is neutral about sustainability, environment and climate change',
 'the sentiment of the provided text is risk about sustainability, environment and climate change',
 'the sentiment of the provided text is opportunity about sustainability, environment and climate change',
 'the sentiment of the provided text is risk about sustainability, environment and climate change',
 'the sentiment of the provided text is opportunity about sustainability, environment and climate change',
 'the sentiment of the provided text is opportunity about sustainability, environment and climate change',
 'the sentiment of the provided text is opportunity abo

In [ ]:
df['gpt-explanations'] = preds

<ipython-input-87-263adef27334>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['gpt-explanations'] = preds


In [ ]:
df[df['gpt-explanations'] == "the sentiment of the provided text is risk about sustainability, environment and climate change"]

text  label  gpt-label  \
4    Finally, there is a reputational risk linked t...      0          0   
6    The Group and its customers are exposed to cli...      0          0   
11   Climate-related risks and opportunities have g...      2          0   
13   An emerging risk is defined as a risk which ha...      1          0   
17   We emit greenhouse gases both directly and ind...      1          0   
..                                                 ...    ...        ...   
302  Nedbank has a Damage to Physical Assets scenar...      1          0   
310  Rising public concern related to climate chang...      1          0   
311  In addition, certain environmental laws assess...      1          0   
312  However, we believe that the data currently di...      1          0   
319  Foreign, federal, state and local regulatory a...      0          0   

                                      gpt-explanations  \
4    the sentiment of the provided text is risk abo...   
6    the sentiment of the provided text is risk abo...   
11   the sentiment of the provided text is risk abo...   
13   the sentiment of the provided text is risk abo...   
17   the sentiment of the provided text is risk abo...   
..                                                 ...   
302  the sentiment of the provided text is risk abo...   
310  the sentiment of the provided text is risk abo...   
311  the sentiment of the provided text is risk abo...   
312  the sentiment of the provided text is risk abo...   
319  the sentiment of the provided text is risk abo...   

                                                points  
4    - Reputational risk for oil companies \n- Poss...  
6    1. The Group and its customers are exposed to ...  
11   Points:\n\n1. Climate-related risks and opport...  
13   Points:\n\n1. Emerging risk is defined as a ri...  
17   Points:\n\n- Greenhouse gases are emitted both...  
..                                                 ...  
302  Points:\n\n- Nedbank has a scenario for physic...  
310  Points:\n1. Rising public concern related to c...  
311  1. Environmental laws can hold current or prev...  
312  Points: \n- Data disclosed by companies does n...  
319  Points:\n\n1. Proposed legislation and regulat...  

[74 rows x 5 columns]

In [ ]:
labels = []

for label in preds:
  if label == "the sentiment of the provided text is risk about sustainability, environment and climate change":
    labels.append(0)
  elif label == "the sentiment of the provided text is neutral about sustainability, environment and climate change":
    labels.append(1)
  elif label == "the sentiment of the provided text is opportunity about sustainability, environment and climate change":
    labels.append(2)


labels

[2,
 2,
 2,
 1,
 0,
 2,
 0,
 2,
 2,
 2,
 1,
 0,
 1,
 0,
 2,
 2,
 2,
 0,
 2,
 1,
 1,
 2,
 2,
 0,
 0,
 0,
 2,
 2,
 2,
 0,
 0,
 2,
 2,
 0,
 1,
 1,
 2,
 2,
 0,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 1,
 2,
 2,
 1,
 2,
 0,
 2,
 2,
 1,
 1,
 1,
 2,
 2,
 2,
 0,
 0,
 2,
 2,
 1,
 2,
 2,
 2,
 0,
 1,
 0,
 2,
 0,
 0,
 2,
 2,
 0,
 1,
 0,
 2,
 1,
 2,
 1,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 0,
 0,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 1,
 2,
 2,
 0,
 2,
 2,
 0,
 1,
 1,
 0,
 2,
 2,
 0,
 2,
 0,
 1,
 1,
 2,
 2,
 1,
 2,
 2,
 1,
 2,
 1,
 0,
 0,
 1,
 2,
 2,
 2,
 1,
 2,
 0,
 0,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 0,
 2,
 1,
 1,
 2,
 0,
 0,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 0,
 0,
 2,
 1,
 2,
 2,
 2,
 2,
 1,
 2,
 0,
 0,
 1,
 1,
 1,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 1,
 1,
 2,
 0,
 2,
 2,
 2,
 0,
 2,
 2,
 1,
 2,
 1,
 0,
 2,
 1,
 0,
 0,
 0,
 2,
 1,
 2,
 0,
 1,
 1,
 0,
 2,
 2,
 1,
 2,
 1,
 2,
 1,
 1,
 2,
 2,
 2,
 2,
 1,
 2,
 2,
 1,
 1,
 2,
 0,
 0,
 2,
 1,
 0,
 1,
 2,
 2,
 2,
 2,
 0,
 2,
 1,
 2,
 1,
 2,
 2,


In [ ]:
df['gpt-label'] = labels

<ipython-input-90-bd7b018092de>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['gpt-label'] = labels


In [ ]:
df

text  label  gpt-label  \
0    Sustainable strategy ‘red lines’ For our susta...      0          2   
1    Verizon’s environmental, health and safety man...      1          2   
2    In 2019, the Company closed a series of transa...      1          2   
3    In December 2020, the AUC approved the Electri...      0          1   
4    Finally, there is a reputational risk linked t...      0          0   
..                                                 ...    ...        ...   
315  Energy Infrastructure revenues of $195 million...      1          1   
316  Financial planning We acknowledge that there a...      1          2   
317  : PCG Group has adopted the PETRONAS Whistlebl...      1          1   
318  Stakeholder dialogue and transparency ⌐ Engagi...      2          2   
319  Foreign, federal, state and local regulatory a...      0          0   

                                      gpt-explanations  \
0    the sentiment of the provided text is opportun...   
1    the sentiment of the provided text is opportun...   
2    the sentiment of the provided text is opportun...   
3    the sentiment of the provided text is neutral ...   
4    the sentiment of the provided text is risk abo...   
..                                                 ...   
315  the sentiment of the provided text is neutral ...   
316  the sentiment of the provided text is opportun...   
317  the sentiment of the provided text is neutral ...   
318  the sentiment of the provided text is opportun...   
319  the sentiment of the provided text is risk abo...   

                                                points  
0    Points:\n1. Sustainable strategy range\n2. Pro...  
1    Points:\n1. Verizon has an environmental, heal...  
2    Points:\n- The Company sold its Canadian fossi...  
3    Points:\n- AUC approved deferral of distributi...  
4    - Reputational risk for oil companies \n- Poss...  
..                                                 ...  
315  - Energy Infrastructure revenues in 2020 were ...  
316  Points:\n-Company intends to integrate climate...  
317  Points:\n- PCG Group has adopted the PETRONAS ...  
318  - Adidas engages openly with stakeholders and ...  
319  Points:\n\n1. Proposed legislation and regulat...  

[320 rows x 5 columns]

In [ ]:
df['gpt-label'].value_counts()

2    179
0     74
1     67
Name: gpt-label, dtype: int64

In [ ]:
df.to_csv("/content/drive/MyDrive/DS-Environment-Project/ChatGPT Results/scikit-llm/chatgpt_climate_sentiment_chained_prompt.csv",index=False)

In [ ]:
# calculate the precision and f1 score for df columns label and prediction
from sklearn.metrics import precision_recall_fscore_support,f1_score
sent_col = 'gpt-label'
print(precision_recall_fscore_support(df['label'], df[sent_col], average='macro'))

# f1 score only
print(f1_score(df['label'], df[sent_col], average='macro'))

# confusion matrix
from sklearn.metrics import confusion_matrix
print(confusion_matrix(df['label'], df[sent_col]))

# performnce report

from sklearn.metrics import classification_report
print(classification_report(df['label'], df[sent_col]))

(0.3103037582734072, 0.2989660810112524, 0.24879227053140096, None)
0.24879227053140096
[[21 21 64]
 [39 34 90]
 [14 12 25]]
              precision    recall  f1-score   support

           0       0.28      0.20      0.23       106
           1       0.51      0.21      0.30       163
           2       0.14      0.49      0.22        51

    accuracy                           0.25       320
   macro avg       0.31      0.30      0.25       320
weighted avg       0.37      0.25      0.26       320

